In [54]:
import matplotlib.pyplot as plt

import torch
from torch import nn
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision import datasets, transforms, models
import cv2
import numpy as np

In [37]:
data_dir = 'data/'
train_dir = data_dir + 'train'
valid_dir = data_dir + 'valid'
test_dir = data_dir + 'test'

batch_size = 20
num_workers = 0

In [38]:
standard_normalization = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                              std=[0.229, 0.224, 0.225])

## Loading Data

In [51]:
data_transforms = {'train': transforms.Compose([transforms.RandomResizedCrop(224),
                                     transforms.RandomHorizontalFlip(),
                                     transforms.ToTensor(),
                                     standard_normalization]),
                   'valid': transforms.Compose([transforms.Resize(256),
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     standard_normalization]),
                   'test': transforms.Compose([transforms.Resize(size=(224,224)),
                                     transforms.ToTensor(), 
                                     standard_normalization])
                  }

In [ ]:
!unzip data.zip

In [180]:
train_data = datasets.ImageFolder(train_dir, transform=data_transforms['train'])
valid_data = datasets.ImageFolder(valid_dir, transform=data_transforms['valid'])
test_data = datasets.ImageFolder(test_dir, transform=data_transforms['test'])

In [181]:
train_loader = torch.utils.data.DataLoader(train_data,
                                           batch_size=batch_size, 
                                           num_workers=num_workers,
                                           shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_data,
                                           batch_size=batch_size, 
                                           num_workers=num_workers,
                                           shuffle=False)
test_loader = torch.utils.data.DataLoader(test_data,
                                           batch_size=batch_size, 
                                           num_workers=num_workers,
                                           shuffle=False)
loaders = {
    'train': train_loader,
    'valid': valid_loader,
    'test': test_loader
}


## Transfer Learning

In [217]:
model = models.densenet121(pretrained=True)

In [218]:
for param in model.parameters():
    param.requires_grad = False

In [219]:
model.classifier = nn.Sequential(nn.Linear(1024, 256),
                                 nn.ReLU(),
                                 nn.Dropout(0.5),
                                 nn.Linear(256, 2))

In [221]:
model

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [222]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.00005)

In [223]:
use_cuda = torch.cuda.is_available()
if use_cuda:
  model = model.cuda()

## Training Function

In [194]:
def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf
    
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        
        ###################
        # train the model #
        ###################
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()

            # initialize weights to zero
            optimizer.zero_grad()
            
            output = model(data)
            
            # calculate loss
            loss = criterion(output, target)
            
            # back prop
            loss.backward()
            
            # grad
            optimizer.step()
            
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            
            if batch_idx % 10 == 0:
                print('Epoch %d, Batch %d loss: %.6f' %
                  (epoch, batch_idx + 1, train_loss))
        
        ######################    
        # validate the model #
        ######################
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            ## update the average validation loss
            output = model(data)
            loss = criterion(output, target)
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.data - valid_loss))

            
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
        
        ## TODO: save the model if validation loss has decreased
        if valid_loss < valid_loss_min:
            torch.save(model.state_dict(), save_path)
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_loss_min,
            valid_loss))
            valid_loss_min = valid_loss
            
    # return trained model
    return model

In [195]:
train(5, loaders, model, optimizer, criterion, use_cuda, 'Model/model.pt')

Epoch 1, Batch 1 loss: 0.697473
Epoch 1, Batch 11 loss: 0.708579
Epoch 1, Batch 21 loss: 0.695220
Epoch 1, Batch 31 loss: 0.690899
Epoch 1, Batch 41 loss: 0.679109
Epoch 1, Batch 51 loss: 0.676600
Epoch 1, Batch 61 loss: 0.672183
Epoch 1, Batch 71 loss: 0.666065
Epoch 1, Batch 81 loss: 0.663098
Epoch 1, Batch 91 loss: 0.656992
Epoch 1, Batch 101 loss: 0.652537
Epoch 1, Batch 111 loss: 0.647361
Epoch 1, Batch 121 loss: 0.642632
Epoch 1, Batch 131 loss: 0.638087
Epoch 1, Batch 141 loss: 0.634754
Epoch 1, Batch 151 loss: 0.631972
Epoch 1, Batch 161 loss: 0.629179
Epoch 1, Batch 171 loss: 0.625964
Epoch 1, Batch 181 loss: 0.623111
Epoch 1, Batch 191 loss: 0.620664
Epoch 1, Batch 201 loss: 0.617398
Epoch 1, Batch 211 loss: 0.613296
Epoch 1, Batch 221 loss: 0.609000
Epoch 1, Batch 231 loss: 0.606856
Epoch: 1 	Training Loss: 0.606136 	Validation Loss: 0.589081
Validation loss decreased (inf --> 0.589081).  Saving model ...
Epoch 2, Batch 1 loss: 0.470571
Epoch 2, Batch 11 loss: 0.539706
Epoch

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

## Test Accuracy

In [225]:
test_loss = 0.0
accuracy = 0.0
with torch.no_grad():
  for inputs, labels in loaders['test']:
      inputs, labels = inputs.to(device), labels.to(device)
      logps = model.forward(inputs)
      batch_loss = criterion(logps, labels)
      
      test_loss += batch_loss.item()
      
      # Calculate accuracy
      
      ps = torch.exp(logps)
      top_p, top_class = ps.topk(1, dim=1)

      equals = top_class == labels.view(*top_class.shape)
      accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

print('Test Accuracy',accuracy, 'Test Loss',test_loss)

Test Accuracy 0.882 	 Test Loss 0.536


# Cheers!